# Classification Test

This notebook runs a single file to test the NCAN classifiers

## Import libraries

In [1]:
# Default libraries
import numpy as np
import mne
import os
import matplotlib.pyplot as plt
import scipy.signal as signal

# Import custom libraries
from functions import data_tools
from functions import processing
from functions import classification
from functions.FeatureExtractorSSVEP import FeatureExtractorCCA as CCA
from functions.FeatureExtractorSSVEP import FeatureExtractorMSI as MSI
from functions.FeatureExtractorSSVEP import FeatureExtractorMEC as MEC

# Magic command to reload libraries
%reload_ext autoreload
# %autoreload 3

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\danie\miniconda3\envs\ssvep-validation\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


## Import and epoch data

In [2]:
# Select file to import
# file = r"C:\Users\danie\OneDrive - University of Calgary\PDF\Proyects\Datasets\easy-on-the-eyes\sub-P004\ses-S001\eeg\sub-P004_ses-S001_task-T1_run-001_eeg.xdf"
file = r"C:\Users\danie\OneDrive - University of Calgary\PDF\Proyects\Datasets\easy-on-the-eyes\sub-P005\ses-S001\eeg\sub-P005_ses-S001_task-T1_run-001_eeg.xdf"

# Import and epoch data
eeg_picks = ["O1","Oz","O2"]
# eeg_picks = "all"
[eeg_time, eeg, srate] = data_tools.read_xdf(file, picks=eeg_picks)
# [eeg_time, eeg, srate] = data_tools.read_xdf(file)
[marker_time, markers] = data_tools.read_xdf_unity_markers(file)
nchans = eeg.shape[0]

# Filter 60 Hz noise
eeg = processing.line_filter(
    eeg = eeg,
    srate = srate,
    f_notch = 60,
    f_order = 4
    )


# Epoch data
[eeg_epochs, epoch_labels] = data_tools.epochs_from_unity_markers(
    eeg_time = eeg_time,
    eeg_data = eeg,
    marker_time = marker_time,
    marker_data = markers
    )

# - Get stimuli and frequencies labels
fixed_labels = data_tools.fix_labels(epoch_labels)          # Fix labels that might be incorrect in pilot trials
dict_of_stimuli = data_tools.get_tvep_stimuli(fixed_labels) # Dictionary of unique stimulus

# Dictionary of stimulus frequencies
dict_of_freqs = {
    0: "9.090909",
    1: "14.28571",
    2: "33.33333"
    }

# - Create array of eeg epochs organized as [stimuli, freq, chans, samples]
eeg_epochs_organized = data_tools.epochs_stim_freq(
    eeg_epochs = eeg_epochs,
    labels = fixed_labels,
    stimuli = dict_of_stimuli,
    freqs = dict_of_freqs,
    mode = "zeropad"
    )

print(f"Type of stimulus\n{dict_of_stimuli}")

Type of stimulus
OrderedDict([(0, ' WoodGrain'), (1, ' Voronoi'), (2, ' Checkerboard'), (3, ' MaxContrast'), (4, ' Static'), (5, ' Worms'), (6, ' MinContrast')])


## Preprocess data

Apply detrend, and temporal and spatial filters.

In [3]:
# Settings
stimuli = 1   # Integer to select the type of SSVEP based on the list above
event_labels = np.array([0,0,0,1,1,1,2,2,2])

# Pick [stimuli] and reshape data to be [freqs*trials, chans, samples]
nsamples = eeg_epochs_organized.shape[-1]
classifier_data = eeg_epochs_organized[stimuli,:,:,:,:].reshape(9,nchans,nsamples)

# Detrend data
detrend_data = signal.detrend(classifier_data, type="constant")

# Apply CSP
csp_data = processing.apply_csp(
    eeg_data = detrend_data,
    labels = event_labels,
    )


Computing rank from data with rank=None
    Using tolerance 5.1e+02 (2.2e-16 eps * 3 dim * 7.7e+17  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.1e+02 (2.2e-16 eps * 3 dim * 7.6e+17  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.2e+02 (2.2e-16 eps * 3 dim * 9.3e+17  max singular value)
    Estimated rank (mag): 3
    MAG: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating covariance using EMPIRICAL
Done.


## Classification

Use different types of classifiers to test their performance

### CCA

In [4]:
# Create classifier
cca = CCA()

# Set classifier settings
# subbands = np.array([[i*3, i*3+2] for i in range(1, 20)])
subbands = np.array((
    (7,11),
    (12,16),
    (31,35),
    (16,20),
    (26,30),
    (64,68)
    ))

cca.setup_feature_extractor(
        harmonics_count = 2,
        targets_frequencies = np.array([float(x) for x in list(dict_of_freqs.values())]),
        sampling_frequency = srate,
        samples_count = eeg_epochs_organized.shape[-1],
        filter_order = 12,
        subbands = subbands
        )

# Run classifier
cca_features = (cca.extract_features(detrend_data))
cca_features = np.squeeze(cca_features)

In [5]:
# Create classifier
# cca2 = CCA2()

# # Set classifier settings
# # subbands = np.array([[i*3, i*3+2] for i in range(1, 20)])
# subbands = np.array((
#     (7,11),
#     (12,16),
#     (31,35),
#     (16,20),
#     (26,30),
#     (64,68)
#     ))

# cca2.setup_feature_extractor(
#         harmonics_count = 2,
#         targets_frequencies = np.array([float(x) for x in list(dict_of_freqs.values())]),
#         sampling_frequency = srate,
#         samples_count = eeg_epochs_organized.shape[-1],
#         filter_order = 12,
#         # subbands = subbands
#         )

# # Run classifier
# cca_features2 = (cca2.extract_features(detrend_data))
# cca_features2 = np.squeeze(cca_features)

In [6]:
# #
# print(f"CCA original {cca_features2} \n")
# print(f"CCA modified {cca_features}")

In [7]:
# np.squeeze(cca_features)
# np.max(np.squeeze(cca_features), axis=1)
# np.squeeze(cca_features)
# np.argmax(np.squeeze(cca_features), axis=1)
np.argmax(np.max(np.squeeze(cca_features), axis=1), axis=1)

array([0, 0, 0, 1, 1, 1, 2, 2, 2], dtype=int64)

In [8]:
# fig, ax = plt.subplots(9,1)


# for e in range(9):
#     epoch = classifier_data[e,:,:]
#     [f, pxx] = signal.welch(epoch, fs = srate, nperseg = srate*7)
#     mask = (f > 5) & (f < 40)
#     ax[e].plot(f[mask], pxx[:,mask].T)
# plt.show()

### MSI

In [9]:
# Create classifier
msi = MSI()

# Set classifier settings
msi.setup_feature_extractor(
        harmonics_count = 2,
        targets_frequencies = np.array([float(x) for x in list(dict_of_freqs.values())]),
        sampling_frequency = srate,
        samples_count = eeg_epochs_organized.shape[-1],
        )

# Run classifier
# msi_data = eeg_epochs_organized[6,:,:,:,:].reshape(9,3,4114)
# msi_data = signal.detrend(msi_data)
msi_features = (msi.extract_features(csp_data))

In [10]:
# np.squeeze(msi_features)
np.argmax(np.squeeze(msi_features), axis=1)

array([0, 0, 0, 1, 1, 1, 0, 0, 0], dtype=int64)

### MEC

In [11]:
# Create classifier
mec = MEC()

# Set classifier settings
mec.setup_feature_extractor(
        harmonics_count = 2,
        targets_frequencies = np.array([float(x) for x in list(dict_of_freqs.values())]),
        sampling_frequency = srate,
        samples_count = eeg_epochs_organized.shape[-1],
        )

# Run classifier
mec_features = (mec.extract_features(classifier_data))


In [12]:
# np.shape(mec_features)
np.squeeze(mec_features)
np.argmax(np.squeeze(mec_features), axis=1)

array([0, 0, 0, 1, 1, 1, 0, 0, 0], dtype=int64)

### Riemmanian geometry + logistic regression

This is made using the [MOABB tutorial](https://moabb.neurotechx.com/docs/auto_examples/plot_cross_subject_ssvep.html#sphx-glr-auto-examples-plot-cross-subject-ssvep-py).

In [13]:
rg_data = eeg_epochs_organized[stimuli,:,:,:,:].transpose((1,2,3,0))

In [16]:
rg_predictions = classification.fb_rg_logreg(
    eeg_data = classifier_data,
    stim_freqs = [float(value) for value in dict_of_freqs.values()],
    eeg_channels = eeg_picks, 
    srate = srate,
    labels = event_labels
    )

print(rg_predictions)

[0 0 0 0 2 2 2 2 2]
